## Lectura de archivo *jpg*

In [ ]:
from PIL import Image # PIL: Python Image Library
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
im = Image.open('./img/grises.jpg')
print(im.format)
print(im.size)
print(im.mode)  ## modo de codificar el color
im

## Escritura de la imagen en disco

In [ ]:
%ls img

In [ ]:
im.save("./img/grises.png", "png")

In [ ]:
%ls img

## Lectura de archivo *png*

In [ ]:
im2 = Image.open('./img/grises.png')
print(im2.format)
print(im2.size)
print(im2.mode)  ## modo de codificar el color
im2

## Conversión de Imagen a Matriz (y viceversa)

* Para pasar de una imagen de PIL a una matriz de Numpy: **matriz = numpy.asarray(imagen)**
* Para pasar de una matriz de Numpy a una imagen de PIL: **imagen = PIL.Image.fromarray(matriz)**

### Ejemplo 1: de imagen a matriz

In [ ]:
matriz_im = np.asarray(im)

print(matriz_im.shape) # la matriz tiene 3 dimensiones porque se necesitan 3 colores para formar cada punto

matriz_im[100:105,100:103]  # muestra algunos elementos de la matriz (un arreglo de 5 x 3)
                            # cada elemento contiene tres números porque es un color RGB

### Ejemplo 2: de matriz a imagen

In [ ]:
imagen = Image.fromarray(matriz_im)
imagen

## Imagen SPECT

In [ ]:
im_spect = Image.open('./img/spect.jpg')
spect = np.asarray(im_spect)
im_spect

In [ ]:
print(spect.shape)
print(spect[100:105,100:103])

## Recorte de imagen (con Numpy)

Podríamos querer fragmentar la imagen original en 4 imagenes diferentes. Una opción es trabajar con la matriz. Como el tamaño de la matriz es de 500x440, sabemos que cada vista ocupa 250x220. Por ende, para obtener la primera vista lateral necesitamos *rebanar* (*slice*) las 220 primeras filas y las 250 primeras columnas.

In [ ]:
recorte_1 = spect[0:220, 0:250]
im_recorte_1 = Image.fromarray(recorte_1)
im_recorte_1

In [ ]:
im_recorte_2 = Image.fromarray(spect[0:220, 250:500])
im_recorte_2

In [ ]:
im_recorte_3 = Image.fromarray(spect[220:440, 0:250])
im_recorte_3

In [ ]:
im_recorte_4 = Image.fromarray(spect[220:440, 250:500])
im_recorte_4

### Definición de una función para automatizar la obtención de un fragmento de la imagen

De paso vemos cómo se "empaqueta" más de una variable en la salida de la función, y cómo se recuperan en diferentes variables las dos salidas de esta función.

In [ ]:
def fragmentar(matriz, fila_inicial, delta_fila, columna_inicial, delta_columna):
    ''' 
    Recibe una matriz con los datos de la imagen
    Calcula un fragmento de la matriz y devuelve el fragmento en dos formatos: matriz e imagen
    '''
    fragmento = matriz[fila_inicial:fila_inicial+delta_fila, columna_inicial:columna_inicial+delta_columna]
    return fragmento, Image.fromarray(fragmento)

In [ ]:
fila_inicial = 20
delta_f = 170

columnna_inicial = 300
delta_c = 140

## obtención de la matriz y de la imagen
frag, imagen_frag = fragmentar(spect, fila_inicial, delta_f, columnna_inicial, delta_c)
imagen_frag

## Recorte de imagen (con PIL)

In [ ]:
cuadro = (300,20,440,190)  # definimos los dos puntos extremos del recorte: (300,20) y (440, 190)
recorte = im_spect.crop(cuadro)
recorte

## Separación de canales (colores) con Numpy

In [ ]:
canal_R = frag[:,:,0]
canal_G = frag[:,:,1]
canal_B = frag[:,:,2]

canales = np.concatenate((canal_R, canal_G, canal_B),axis=1)
Image.fromarray(canales)

## Separación de canales (colores) con PIL

In [ ]:
r, g, b = recorte.split()

In [ ]:
b

## Histograma de una imagen monocromática  con PIL

Podemos obtener el histograma de una imagen monocromática con la función histogram(), que devuelve una lista con 256 elementos (1 por cada nivel de gris). Recordemos primero cuál es la imagen de la cual partimos:

In [ ]:
r

Ahora sí calculamos el histograma y mostramos sus características. 

In [ ]:
hist_r = r.histogram()
print("tipo de datos del histograma: ", type(hist_r))
print("cantidad de elementos del histograma: ", len(hist_r))
plt.plot(hist_r);   # el punto y coma evita que aparezca información del objeto que no necesitamos

## Histograma de una imagen en color con PIL

Si la imagen contiene los tres canales de color, la función histogram() devuelve una sola lista con 768 elementos (3 * 256). Nuevamente, recordemos la imagen desde la cual partimos:

In [ ]:
recorte

Ahora sí obtenemos los tres histogramas, que aparecen concatenados, uno a continuación del otro.

In [ ]:
plt.plot(recorte.histogram());

Puedo separar los 3 histogramas rebanando la lista y graficándolos superpuestos: 

In [ ]:
plt.plot(recorte.histogram()[0:256])
plt.plot(recorte.histogram()[256:512])
plt.plot(recorte.histogram()[512:]);

El pico inicial corresponde al negro, y los valores cercanos a 256 corresponden al blanco. Como gran parte de la imagen tiene negro entonces por eso aparece un pico en ese valor. Grafiquemos el histograma pero salteando los valores cercanos a estos dos extremos (negro y blanco), para ver con más detalle los valores intermedios.

In [ ]:
offset = 5

plt.plot(recorte.histogram()[offset:256-offset])
plt.plot(recorte.histogram()[256+offset:512-offset])
plt.plot(recorte.histogram()[512+offset:-offset])

### Histograma con Numpy

Recordemos que la matriz (full color) está almacenada en **matriz_im** y en **frag**. En este caso el argumento *bins* permite determinar en cuantos segmentos dividir el dominio (en vez de que el número de segmentos sea el número total de grises).

In [ ]:
histograma, bordes = np.histogram(frag, bins=10)
histograma

Podría graficar directamente con **plt.plot(histograma)** pero cambiarían los números en el eje x. El array *bordes* tiene un elemento más que el histograma, por eso elimino el último.

In [ ]:
print("Cantidad de elementos en bordes: ", len(bordes))
print("Cantidad de elementos en histograma: ", len(histograma))
plt.plot(bordes[0:-1], histograma);

### Histograma con Matplotlib

En este caso se usa la función **hist()** de Matplotlib. Antes se debe "desenrollar" la matriz en un vector unidimensional, lo cual se logra con la función **ravel()**. Por defecto el dominio se divide en 10 segmentos.

In [ ]:
plt.hist(frag.ravel());

Pero también se puede dividir en más (o menos) segmentos:

In [ ]:
plt.hist(frag.ravel(), bins=50);

## Luminancia

Las imágenes en color a se pueden convertir a escala de grises con la función de luminancia. Esta función tiene en cuenta nuestra característica de percepción visual, que no es lineal: por ejemplo, el gris medio que nuestro sistema visual identifica a la misma distancia del blanco que del negro, no se corresponde con el punto medio del rojo, verde y azul.

In [ ]:
foto = Image.open('./img/parana.jpg')
foto

In [ ]:
foto_gris = foto.convert('L')
foto_gris

In [ ]:
plt.plot(foto_gris.histogram());

## Operaciones con la imagen

### Binarización

Consiste en la conversión de la imagen a blanco y negro. Para ello se obtiene una matriz con las mismas dimensiones de la matriz original, pero conteniendo sólo valores booleanos (verdaderos y falsos). Como son dos valores, se interpreta como blanco y negro al convertir los datos en imágenes.

Es interesante notar que la operación se efectúa sin tener que iterar a través de cada elemento de la matriz con un bucle **for**. La operación de comparación se realiza elemento a elemento con una sintaxis muy sencilla.

In [ ]:
original = np.asarray(foto_gris)   # convertimos la imagen a una matriz
binarizada = original < 100

print(binarizada)

foto_bin = Image.fromarray(binarizada)
foto_bin

In [ ]:
plt.plot(foto_bin.histogram());

### Umbralizado

Se convierten en negro los grises más oscuros, por encima de un umbral, y se mantiene la luminosidad del resto.

In [ ]:
umbral = 120
umbralizada = (original > umbral) * original

print(umbralizada)

foto_umbralizada = Image.fromarray(umbralizada)
foto_umbralizada

In [ ]:
plt.plot(foto_umbralizada.histogram());